## Feature Engineering

El procedimiento llevado a cabo aquí se enfoca en calcular el sentimiento de cada revisión para luego asignarle una puntuación de análisis de sentimientos según esa evaluación y si se recomienda o no. Antes de realizar el análisis de sentimientos, se aplica el stemming (PorterStemmer) definiendo la función apply_stemming para aplicarlo a la columna 'review' del DataFrame. Esto reduce las palabras a su forma base para capturar mejor el significado general y reducir la dimensionalidad del texto.

*Importamos las librerías necesarias para realizar el análisis.*

In [3]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer  # Importar el stemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Importar el lematizador
import nltk

In [4]:
nltk.download('vader_lexicon')  # Descargar el modelo vader_lexicon de NLTK
nltk.download('punkt')  # Descargar el tokenizer de NLTK
nltk.download('wordnet')  # Descargar WordNet para lematización

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Cargar el dataset:\
*Cargamos el dataset en un DataFrame de pandas para poder manipularlo y realizar el análisis de sentimiento.*

In [5]:
df = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\reviews_clean.parquet')

*Hacemos una copia para realizar las operaciones, mientras resguardamos los datos en el archivo original.*

In [6]:
df_copy = df.copy()
df_copy

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,:D


*Limpiamos el dataset previa al análisis.*

In [7]:
# Eliminar filas con valores nulos en la columna 'review'
# df = df.dropna(subset=['review'])

# Reemplazar expresiones
df_copy['review'] = df_copy['review'].replace(['10/10', '100/100'], 'Excelent', regex=True)

# Transformar todos los datos a tipo string
df_copy['review'] = df_copy['review'].astype(str)

*Inicializamos el Analizador de Intensidad de Sentimiento (Sentiment Intensity Analyzer), el stemmer y el lematizador*

*Creamos una instancia para cada proceso.*

In [8]:
# Inicializar el analizador de intensidad de sentimiento de NLTK
sid = SentimentIntensityAnalyzer()

# Inicializar el stemmer
stemmer = PorterStemmer()

# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()  

*Aplicación de Stemming a la Columna 'review' del DataFrame:*\
Utilizamos el algoritmo de Stemming de NLTK para reducir las palabras a su forma base en la columna 'review' de nuestro DataFrame.

In [9]:
# Definir la Función de Aplicación del Stemming
def apply_stemming(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Aplicar la función de stemming a la columna 'review' del DataFrame
df_copy['review'] = df_copy['review'].apply(apply_stemming)

*Aplicación de Lematización a la Columna 'review' del DataFrame:*\
Utilizamos la función word_tokenize para dividir el texto en palabras. Después, aplicamos la lematización a cada palabra utilizando lemmatizer.lemmatize(word). Para terminar, arrgregamos las palabras lematizadas en una cadena utilizando ' '.join(lemmatized_words) y asignmaos el resultado a la columna 'review' del DataFrame df_copy.

In [10]:
# Definir la Función de Aplicación de la Lematización
def apply_lemmatization(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Aplicar la función de lematización a la columna 'review' del DataFrame
df_copy['review'] = df_copy['review'].apply(apply_lemmatization)

*Aplicación del análisis de sentimiento:*\
Creamos una función. Esta función tomará una reseña y aplicará el análisis de sentimiento, devolviendo el valor correspondiente según la escala especificada (malo='0', neutral='1' y positivo='2'). La función devolverá '1' en caso de que la reseña está ausente. Luego se la aplica al DataFrame.

In [11]:
# Definir la Función de Análisis de Sentimiento
def get_sentiment_score(text, recommend):
    if pd.isnull(text):
        return 1
    sentiment_score = sid.polarity_scores(text)['compound']
    if recommend:
        sentiment_score += 0.5
    else:
        sentiment_score -= 0.5
    if sentiment_score <= -0.05:
        return 0
    elif sentiment_score >= 0.05:
        return 2
    else:
        return 1

# Aplicar la función get_sentiment_score a la columna de reseñas (review) para crear la nueva columna 'sentiment_analysis'.
df_copy['sentiment_analysis'] = df_copy.apply(lambda row: get_sentiment_score(row['review'], row['recommend']), axis=1)

df_copy
df_copy.drop(columns=['review'], inplace=True)

In [12]:
df_copy

,user_id,user_url,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,2
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,2
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,2
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,2


*Estudiamos la correlación entre the sentimientos expresados en las reseñas y las recomendaciones proporcionadas por los usuarios.*\

*Impremimos el recuento de las reseñas que presentan discrepancias entre el sentimiento expresado y la calificación proporcionada por los usuarios, detallando la cantidad de reseñas con sentimiento negativo y calificación positiva, así como la cantidad de reseñas con sentimiento positivo y calificación negativa.*

In [13]:
# Reseñas con sentimiento positivo pero con una recomendación negativa.
positive_negative = df_copy[(df_copy['sentiment_analysis'] == 2) & (df_copy['recommend'] == False)]

# Reseñas con sentimiento negativo pero con una recomendación positiva.
negative_positive = df_copy[(df_copy['sentiment_analysis'] == 0) & (df_copy['recommend'] == True)]

# Conteo de reseñas con sentimiento negativo y recomendación positiva.
count_negative_positive = len(negative_positive)

# Conteo de reseñas con sentimiento positivo y recomendación negativa.
count_positive_negative = len(positive_negative)

print("Conteo de reseñas con sentimiento negativo y recomendación positiva:", count_negative_positive)
print("Conteo de reseñas con sentimiento positivo y recomendación negativa:", count_positive_negative)

Conteo de reseñas con sentimiento negativo y recomendación positiva: 2904
Conteo de reseñas con sentimiento positivo y recomendación negativa: 1323


*Calculamos el porcentaje de errores en relación con el total de registros para determinar el margen de error del análisis de sentimientos.*

In [14]:
total_mistakes = 2904 + 1323
total_records = 59333
percent_mistakes = (total_mistakes / total_records) * 100

print("Porcentaje de errores:", percent_mistakes)

Porcentaje de errores: 7.124197326951275


*(Opcional) Guardamos el DataFrame actualizado en un nuevo archivo CSV.*

In [26]:
df_copy.to_csv(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM', index=False)

In [ ]:
*(Opcional) Guardamos el DataFrame actualizado en un nuevo archivo parquet.*

In [16]:
df_copy.to_parquet('reviews_clean_sentiment.parquet', index=False)